In [4]:
#필요한 모듈을 불러옵니다.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from FM import FM

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten
from tensorflow.keras.layers import Dense, Concatenate, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam, Adamax
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model

import pandas as pd
import numpy as np
import time
from RMSE import TENSOR_RMSE, RMSE

In [2]:
#데이터를 불러옵니다.
columns = ['ISBN', "User-ID", 'Book-Rating', 'Book-Author', 'Country', 'Age labeling']
ratings = pd.read_csv('./merged_data_revised2.csv', usecols=columns)
ratings = ratings[ratings['Book-Rating']!=0]
ratings.head()

,ISBN,User-ID,Book-Author,Book-Rating,Country,Age labeling
0,33652,2923,Sue Harrison,10,germany,30대
1,35215,1812,Jean P. Sasson,7,germany,30대
2,36016,5112,Dean Koontz,8,germany,20대
3,36016,630,Dean Koontz,8,switzerland,40대
4,37111,4084,Rita Mae Brown,9,germany,20대


In [3]:
#데이터에 string이 섞여 있어 모두 int 인덱스 형태로 변경해줍니다. 
item_dict = {}
user_dict = {}
author_dict = {}
country_dict = {}
age_dict = {}
def convert(x, dic):
    if x in dic:
        return dic[x]
    else:
        dic[x] = len(dic)
        return dic[x]

In [4]:
ratings['Book-Author'] = ratings['Book-Author'].apply(lambda x : convert(x, author_dict))
ratings['Country'] = ratings['Country'].apply(lambda x : convert(x, country_dict))
ratings['Age labeling'] = ratings['Age labeling'].apply(lambda x : convert(x, age_dict))
ratings['User-ID'] = ratings['User-ID'].apply(lambda x : convert(x, user_dict))
ratings['ISBN'] = ratings['ISBN'].apply(lambda x : convert(x, item_dict))
L1 = len(author_dict)
L2 = len(country_dict)
L3 = len(age_dict)

In [5]:
#FM 알고리즘에 적합한 형태로 데이터를 인코딩합니다.
# User encoding
User_dict = {}
for i in set(ratings['User-ID']):
    User_dict[i] = len(User_dict)
n_user = len(User_dict)
# Item encoding
Item_dict = {}
start_point = n_user
for i in set(ratings['ISBN']):
    Item_dict[i] = start_point + len(Item_dict)
n_item = len(Item_dict)
start_point += n_item
# Country encoding
Country_dict = {}
for i in set(ratings['Country']):
    Country_dict[i] = start_point + len(Country_dict)
n_country = len(Country_dict)
start_point += n_country
# Age encoding
Age_dict = {}
for i in set(ratings['Age labeling']):
    Age_dict[i] = start_point + len(Age_dict)
n_age = len(Age_dict)
start_point += n_age
num_x = start_point

In [6]:
FM_results = []
fm_model_list = []
# FM 알고리즘을 다섯번 실행
for j in range(5):
    tic = time.time()
    print(f'{j+1}번째 데이터 인코딩 실행')
    ratings = shuffle(ratings, random_state=j)
    data = []
    y = []
    w0 = np.mean(ratings['Book-Rating'])
    for k in range(len(ratings)):
        case = ratings.iloc[k]
        x_index = []
        x_value = []
        x_index.append(User_dict[case['User-ID']])     # User id encoding
        x_value.append(1.)
        x_index.append(Item_dict[case['ISBN']])    # Item id encoding
        x_value.append(1.)
        x_index.append(Country_dict[case['Country']])       # Country id encoding
        x_value.append(1.)
        x_index.append(Age_dict[case['Age labeling']])       # Age id encoding
        x_value.append(1.)

        data.append([x_index, x_value])
        y.append(case['Book-Rating'] - w0)
        if (k % 10000) == 0:
            print('Encoding ', k, ' cases...')

    #FM 알고리즘을 실행합니다.
    print()
    print(f'{j+1}번째 FM 알고리즘 실행')
    K = 200
    fm1 = FM(num_x, K, data, y, alpha=0.00013, beta=0.008, train_ratio=0.70, iterations=900, tolerance=0.0001, l2_reg=True, verbose=True)
    result = fm1.test()

    fm_model_list.append(fm1)
    FM_results.append(result)
    tac = time.time()
    print(f'걸린 시간 : {tac - tic}')

1번째 데이터 인코딩 실행
Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...
Encoding  30000  cases...

1번째 FM 알고리즘 실행
Iteration: 10 ; Train RMSE = 1.801271 ; Test RMSE = 1.797260
Iteration: 20 ; Train RMSE = 1.786205 ; Test RMSE = 1.786987
Iteration: 30 ; Train RMSE = 1.770443 ; Test RMSE = 1.776806
Iteration: 40 ; Train RMSE = 1.751173 ; Test RMSE = 1.764762
Iteration: 50 ; Train RMSE = 1.723870 ; Test RMSE = 1.748160
Iteration: 60 ; Train RMSE = 1.683674 ; Test RMSE = 1.726110
Iteration: 70 ; Train RMSE = 1.630545 ; Test RMSE = 1.702452
Iteration: 80 ; Train RMSE = 1.568518 ; Test RMSE = 1.682070
Iteration: 90 ; Train RMSE = 1.501682 ; Test RMSE = 1.667527
Iteration: 100 ; Train RMSE = 1.433222 ; Test RMSE = 1.659372
Iteration: 110 ; Train RMSE = 1.364807 ; Test RMSE = 1.656747
109 1.6567472940445758
걸린 시간 : 196.2196991443634
2번째 데이터 인코딩 실행
Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...
Encoding  30000  cases...

2번째 FM 알고리즘 실행
Iteration: 10 ; 

In [7]:
# FM 알고리즘에서 사용한 Train data와 Test data를 DL에서도 똑같이 사용하기 위해 사용한 코드입니다.
ratings_train_list = []
ratings_test_list = []
train_country_list = []
test_country_list = []
train_age_list = []
test_age_list = []
mu_list = []
# 모델별로 Train - Test 데이터가 다르므로 각각 모델별로 데이터 추출
for model in fm_model_list:
    train_user_idx = []
    train_item_idx = []
    for i in model.train_x:
        train_user_idx.append(i[0][0])
        for idx, item in Item_dict.items():
            if item == i[0][1]:
                train_item_idx.append(idx)

    train_frame_list = []
    for user, item in zip(train_user_idx, train_item_idx):
        train_frame_list.append(ratings[(ratings['ISBN'] == item) & (ratings['User-ID'] == user)])
    ratings_train = pd.concat(train_frame_list)

    test_user_idx = []
    test_item_idx = []
    for i in model.test_x:
        test_user_idx.append(i[0][0])
        for idx, item in Item_dict.items():
            if item == i[0][1]:
                test_item_idx.append(idx)

    test_frame_list = []
    for user, item in zip(test_user_idx, test_item_idx):
        test_frame_list.append(ratings[(ratings['ISBN'] == item) & (ratings['User-ID'] == user)])
    ratings_test = pd.concat(test_frame_list)

    train_country = ratings_train['Country']
    train_age = ratings_train['Age labeling']

    test_country = ratings_test['Country']
    test_age = ratings_test['Age labeling']

    mu = ratings_train['Book-Rating'].mean()    # 전체 평균 

    ratings_train_list.append(ratings_train)
    ratings_test_list.append(ratings_test)
    train_country_list.append(train_country)
    test_country_list.append(test_country)
    train_age_list.append(train_age)
    test_age_list.append(test_age)
    mu_list.append(mu)


In [8]:
K = 200                             # Latent factor 수 
reg = 0.0001
M = len(ratings['User-ID']) + 1       # Number of users
N = len(ratings['ISBN']) + 1      # Number of movies

In [53]:
#딥러닝 학습을 다섯번 수행
dl_model_list = []
num = 1
for ratings_train, ratings_test, train_country, test_country, train_age, test_age, mu in zip(ratings_train_list, ratings_test_list, train_country_list, test_country_list, train_age_list, test_age_list, mu_list):
    user = Input(shape=(1, ))
    item = Input(shape=(1, ))
    P_embedding = Embedding(M, K, embeddings_regularizer=l2(reg))(user)
    Q_embedding = Embedding(N, K, embeddings_regularizer=l2(reg))(item)
    user_bias = Embedding(M, 1, embeddings_regularizer=l2(reg))(user)
    item_bias = Embedding(N, 1, embeddings_regularizer=l2(reg))(item)

    P_embedding = Flatten()(P_embedding)
    Q_embedding = Flatten()(Q_embedding)
    user_bias = Flatten()(user_bias)
    item_bias = Flatten()(item_bias)

    country = Input(shape=(1, ))
    country_embedding = Embedding(L2, 3, embeddings_regularizer=l2())(country)
    country_layer = Flatten()(country_embedding)
    age = Input(shape=(1, ))
    age_embedding = Embedding(L3, 3, embeddings_regularizer=l2())(age)
    age_layer = Flatten()(age_embedding)

    R = Concatenate()([P_embedding, Q_embedding, user_bias, item_bias, country_layer, age_layer])
    R = Dense(2048)(R)
    R = Activation('linear')(R)
    R = Dense(512)(R)
    R = Activation('linear')(R)
    R = Dense(1)(R)
    
    model = Model(inputs=[user, item, country, age], outputs=R)
    model.compile(
      loss=TENSOR_RMSE,
      optimizer=SGD(lr=0.001, momentum=0.9),
      #optimizer=Adamax(lr=0.0005),
      metrics=[TENSOR_RMSE]
      )
    result = model.fit(
    x=[ratings_train['User-ID'].values, ratings_train['ISBN'].values,
        train_country.values, train_age.values],
    y=ratings_train['Book-Rating'].values - mu,
    epochs=100,
    batch_size=256,
    validation_data=(
      [ratings_test['User-ID'].values, ratings_test['ISBN'].values,
      test_country, test_age],
      ratings_test['Book-Rating'].values - mu
      ),
    callbacks=[EarlyStopping(monitor='val_loss',
                                           patience=3),
            ModelCheckpoint(f'dl_{num}.h5', monitor='val_loss', mode='min', save_best_only=True)]
      )
    dl_model_list.append(model)
    num += 1

/home/lhmlhm1111/.local/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/100
98/98 [==============================] - 6s 57ms/step - loss: 3.0188 - TENSOR_RMSE: 1.8280 - val_loss: 3.0097 - val_TENSOR_RMSE: 1.8210
Epoch 2/100
98/98 [==============================] - 5s 55ms/step - loss: 3.0180 - TENSOR_RMSE: 1.8314 - val_loss: 3.0087 - val_TENSOR_RMSE: 1.8205
Epoch 3/100
98/98 [==============================] - 5s 56ms/step - loss: 3.0167 - TENSOR_RMSE: 1.8366 - val_loss: 3.0077 - val_TENSOR_RMSE: 1.8200
Epoch 4/100
98/98 [==============================] - 5s 55ms/step - loss: 3.0161 - TENSOR_RMSE: 1.8260 - val_loss: 3.0067 - val_TENSOR_RMSE: 1.8195
Epoch 5/100
98/98 [==============================] - 5s 55ms/step - loss: 3.0141 - TENSOR_RMSE: 1.8214 - val_loss: 3.0055 - val_TENSOR_RMSE: 1.8188
Epoch 6/100
98/98 [==============================] - 5s 52ms/step - loss: 3.0130 - TENSOR_RMSE: 1.8252 - val_loss: 3.0044 - val_TENSOR_RMSE: 1.8181
Epoch 7/100
98/98 [==============================] - 5s 54ms/step - loss: 3.0115 - TENSOR_RMSE: 1.8250 - val_los

In [56]:
dl_model_list = [load_model(f'dl_{i}.h5', custom_objects={'TENSOR_RMSE': TENSOR_RMSE}) for i in range(1,6)]

In [57]:
# FM
def recommender0(recomm_list, fm):
    recommendations = []
    for i in range(len(recomm_list)):
        data = [User_dict[recomm_list[i][1]], Item_dict[recomm_list[i][0]], 
                Country_dict[recomm_list[i][4]], Age_dict[recomm_list[i][5]]]
        recommendations.append(fm.predict(data, [1,1,1,1]))
    return recommendations

# 딥러닝
def recommender1(recomm_list, model, mu):
    book_ids = recomm_list[:,0]
    user_ids = recomm_list[:,1]
    #author_ids = recomm_list[:,2]
    country_ids = recomm_list[:,4]
    age_id = recomm_list[:,5]
    recommendations = model.predict([book_ids, user_ids, country_ids, age_id], batch_size=256) + mu
    return recommendations

In [58]:
result0_list = []
result1_list = []
for ratings_test, fm1, model, mu in zip(ratings_test_list, fm_model_list, dl_model_list, mu_list):
    recomm_list = np.array(ratings_test)
    result0 = recommender0(recomm_list, fm1)
    result0 = result0 + np.array([ratings_test['Book-Rating'].mean()]*len(result0))
    result1 = np.ravel(recommender1(recomm_list, model, mu))

    result0_list.append(result0)
    result1_list.append(result1)

In [59]:
RMSE_list = []
fm_RMSE_list = []
dl_RMSE_list = []
for result0, result1, ratings_test in zip(result0_list, result1_list, ratings_test_list):
    weight = [0.9, 0.1]
    predictions = []
    for i, number in enumerate(result0):
        predictions.append(result0[i] * weight[0] + result1[i] * weight[1])
    RMSE_list.append(RMSE(ratings_test['Book-Rating'], predictions))
    fm_RMSE_list.append(RMSE(ratings_test['Book-Rating'], result0))
    dl_RMSE_list.append(RMSE(ratings_test['Book-Rating'], result1))

In [60]:
print(np.mean(RMSE_list)) #하이브리드 결과
print(np.mean(fm_RMSE_list)) #fm 결과
print(np.mean(dl_RMSE_list)) #딥러닝 결과

1.6627632101877752
1.6627816560677602
1.9478432313864875
